In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd "/content/drive/MyDrive/HP3/LLM/SLM_Experiment/"

/content/drive/MyDrive/HP3/LLM/SLM_Experiment


# 1.Load Dataset Resume-JD

In [ ]:
import pandas as pd
import json
from pathlib import Path

def create_realtest_df(root_path):
    root = Path(root_path)

    # Load Matching_Result.json
    with open(root / "Matching_Result.json", "r", encoding="utf-8") as f:
        data = json.load(f)

    rows = []
    for item in data:
        cv_name = item["CV"] + ".txt"
        jd_name = item["JD"] + ".txt"
        score = item["Score"]

        cv_path = root / "CV" / cv_name
        jd_path = root / "JD" / jd_name

        # Đọc nội dung file CV và JD
        if cv_path.exists() and jd_path.exists():
            with open(cv_path, "r", encoding="utf-8") as f:
                cv_text = f.read().strip()
            with open(jd_path, "r", encoding="utf-8") as f:
                jd_text = f.read().strip()

            rows.append({
                "CV_Text": cv_text,
                "JD_Text": jd_text,
                "Score": score
            })
        else:
            print(f"⚠️ File missing: {cv_name} or {jd_name}")

    df_realtest = pd.DataFrame(rows)
    return df_realtest


df_real = create_realtest_df("../Data_Collector_11_9")

df_train = pd.read_csv("./resume_jd_kaggle_dataset/train.csv")
df_test = pd.read_csv("./resume_jd_kaggle_dataset/test.csv")
df_test

,job_description,resume,match_score,data_id
0,Product Manager needed with experience in Scru...,"Experienced professional skilled in Agile, Use...",8,id_6252
1,Product Manager needed with experience in Prod...,"Experienced professional skilled in life, Scru...",4,id_4684
2,Data Scientist needed with experience in Machi...,"Experienced professional skilled in night, alt...",4,id_1731
3,Data Analyst needed with experience in Tableau...,"Experienced professional skilled in book, SQL,...",4,id_4742
4,"ML Engineer needed with experience in Python, ...",Experienced professional skilled in Computer V...,8,id_4521
...,...,...,...,...
2995,"Data Analyst needed with experience in SQL, Ex...","Experienced professional skilled in SQL, Excel...",6,id_8014
2996,Product Manager needed with experience in Agil...,Experienced professional skilled in Stakeholde...,6,id_1074
2997,Product Manager needed with experience in Stak...,"Experienced professional skilled in might, fam...",4,id_3063
2998,ML Engineer needed with experience in Computer...,"Experienced professional skilled in Keras, com...",2,id_6487


# 2.Create Resume-JD Embedding

In [ ]:
!ls

res_output_merge       resume_jd_kaggle_dataset  slm_cv_jd_seperation.ipynb
res_output_seperation  slm_cv_jd_merge.ipynb


In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
model

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False, 'architecture': 'BertModel'})
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)

In [ ]:
print(model[0].auto_model.config)

BertConfig {
  "architectures": [
    "BertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "dtype": "float32",
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 384,
  "initializer_range": 0.02,
  "intermediate_size": 1536,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.57.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}



In [ ]:
# model_list = [
#     "sentence-transformers/all-MiniLM-L6-v2",
#     "sentence-transformers/paraphrase-MiniLM-L6-v2",

#     "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
#     "sentence-transformers/distiluse-base-multilingual-cased-v2",

#     "sentence-transformers/all-MiniLM-L12-v2",
#     "sentence-transformers/all-mpnet-base-v2",
#     "sentence-transformers/paraphrase-mpnet-base-v2",
#     "sentence-transformers/multi-qa-mpnet-base-dot-v1",

#     "sentence-transformers/stsb-xlm-r-multilingual",
#     "sentence-transformers/all-distilroberta-v1"
# ]

import os
from sentence_transformers import SentenceTransformer
import pandas as pd
from multiprocessing import Pool

def create_bert_embeddings_df(input_df,
                              job_col="job_description",
                              resume_col="resume",
                              model_name="sentence-transformers/all-MiniLM-L6-v2",
                              batch_size=32,
                              output_path=None):
    # Load model
    model = SentenceTransformer(model_name)

    # ✅ Ghép text trước khi encode
    combined_texts = (input_df[job_col].astype(str) + " [SEP] " + input_df[resume_col].astype(str)).tolist()

    # ✅ Encode chỉ 1 lần
    combined_embeds = model.encode(combined_texts, batch_size=batch_size, convert_to_tensor=True, show_progress_bar=True)

    # Add back to dataframe
    df_output = input_df.copy()
    df_output["combined_embedding"] = combined_embeds.tolist()

    # Save if needed
    if output_path:
        df_output.to_pickle(output_path)  # pickle để giữ tensor object

    return df_output

# df_train_embed = create_bert_embeddings_df(df_train,
#                                            job_col="job_description",
#                                            resume_col="resume",
#                                            batch_size=32,
#                                            output_path="./res_output_merge/train_with_combined_embeddings.pkl")
# df_train_embed.to_csv("./res_output_merge/df_train_embed.csv", index=False)

# df_test_embed = create_bert_embeddings_df(df_test,
#                                            job_col="job_description",
#                                            resume_col="resume",
#                                            batch_size=32,
#                                            output_path="./res_output_merge/test_with_combined_embeddings.pkl")
# df_test_embed.to_csv("./res_output_merge/df_test_embed.csv", index=False)

df_real_embed = create_bert_embeddings_df(df_real,
                                          job_col="JD_Text",
                                          resume_col="CV_Text",
                                          batch_size=32,
                                          output_path="./res_output_merge/87_pairs_with_ccombined_embeddings.pkl")
df_real_embed.to_csv("./res_output_merge/df_real_embed.csv", index=False)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
import numpy as np
import pandas as pd

df_train_embed = pd.read_csv("./res_output_merge/df_train_embed.csv")
df_test_embed = pd.read_csv("./res_output_merge/df_test_embed.csv")

len(df_real_embed['combined_embedding'][0:1].values[0])

384

# 3.Model training - Ridge Regression


In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.decomposition import PCA
from sklearn.linear_model import Ridge, RidgeCV, LassoCV, Lasso
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

# XGBoost (optional)
try:
    from xgboost import XGBRegressor
except Exception:
    XGBRegressor = None
    print("⚠️ xgboost not available. Install with `pip install xgboost` to use XGBoost models.")


# -------------------------
# Utils
# -------------------------
def prepare_feature_matrix(df, emb_col="combined_embedding"):
    X = np.stack(df[emb_col].values)
    return X

def compute_accuracy(df, label_col="match_score", pred_col="predicted_match_score"):
    acc1 = (np.abs(df[label_col] - df[pred_col]) <= 1).mean()
    acc2 = (np.abs(df[label_col] - df[pred_col]) <= 2).mean()
    acc3 = (np.abs(df[label_col] - df[pred_col]) <= 3).mean()
    return acc1, acc2, acc3


def plot_score_distribution(df, model_name, chart_output_folder, label_col="match_score", pred_col="predicted_match_score", suffix=""):
    os.makedirs(chart_output_folder, exist_ok=True)
    plt.figure(figsize=(10, 6))
    plt.hist(df[label_col], bins=10, alpha=0.6, label='Labels')
    plt.hist(df[pred_col], bins=10, alpha=0.6, label='Predicts')
    plt.title(f"Score Distribution - {model_name} {suffix}", fontsize=14)
    plt.xlabel("Score", fontsize=12)
    plt.ylabel("Count", fontsize=12)
    plt.legend()
    plt.grid(axis='y', linestyle='--', alpha=0.6)
    safe_name = model_name.replace('/', '_')
    save_path = os.path.join(chart_output_folder, f"{safe_name}{('_'+suffix) if suffix else ''}.png")
    plt.savefig(save_path)
    plt.close()
    print(f"✅ Saved chart to {save_path}")


# -------------------------
# Trainers for each method
# -------------------------
def train_and_eval(df_train,
                   df_test_embed,
                   method,
                   emb_col="combined_embedding",
                   label_col="match_score",
                   test_size=0.2,
                   random_state=42,
                   n_components=200,
                   chart_folder="./res_output/charts"):
    """
    Train model hồi quy với combined_embedding.
    """
    assert method in ["pca_ridge", "ridge", "pca_lasso", "pca_svr", "pca_randomforest", "pca_xgboost"], \
        f"Unknown method: {method}"

    # Feature matrix & label
    X_all = prepare_feature_matrix(df_train, emb_col)
    y_all = df_train[label_col].values.astype(float)

    # Split
    X_train, X_val, y_train, y_val = train_test_split(X_all, y_all, test_size=test_size, random_state=random_state)

    # External test
    X_external = prepare_feature_matrix(df_test_embed, emb_col)
    y_external = df_test_embed[label_col].values.astype(float)

    # Scale
    scaler = StandardScaler()
    X_train_std = scaler.fit_transform(X_train)
    X_val_std = scaler.transform(X_val)
    X_external_std = scaler.transform(X_external)

    pca = None
    if method.startswith("pca_"):
        pca = PCA(n_components=n_components)
        X_train_proc = pca.fit_transform(X_train_std)
        X_val_proc = pca.transform(X_val_std)
        X_external_proc = pca.transform(X_external_std)
        print(f"🔷 [{method}] PCA n_components={n_components} explained_var_sum={pca.explained_variance_ratio_.sum():.4f}")
    else:
        X_train_proc, X_val_proc, X_external_proc = X_train_std, X_val_std, X_external_std

    # -------------------------
    # Model training
    # -------------------------
    if method in ("pca_ridge", "ridge"):
        alphas = np.logspace(-3, 3, 13)
        ridge_cv = RidgeCV(alphas=alphas, scoring="neg_mean_squared_error", cv=5)
        ridge_cv.fit(X_train_proc, y_train)
        model = Ridge(alpha=ridge_cv.alpha_, random_state=random_state)
        model.fit(X_train_proc, y_train)
        print(f"🔧 [{method}] Ridge trained. alpha={ridge_cv.alpha_}")
    elif method == "pca_lasso":
        lasso_cv = LassoCV(cv=5, n_alphas=50, random_state=random_state, max_iter=5000)
        lasso_cv.fit(X_train_proc, y_train)
        model = Lasso(alpha=lasso_cv.alpha_, max_iter=10000)
        model.fit(X_train_proc, y_train)
        print(f"🔧 [{method}] Lasso trained. alpha={lasso_cv.alpha_}")
    elif method == "pca_svr":
        model = SVR()
        model.fit(X_train_proc, y_train)
        print(f"🔧 [{method}] SVR trained.")
    elif method == "pca_randomforest":
        model = RandomForestRegressor(n_estimators=200, random_state=random_state, n_jobs=-1)
        model.fit(X_train_proc, y_train)
        print(f"🔧 [{method}] RandomForest trained.")
    elif method == "pca_xgboost":
        if XGBRegressor is None:
            raise ImportError("xgboost not installed.")
        model = XGBRegressor(n_estimators=200, verbosity=0, random_state=random_state, n_jobs=-1)
        model.fit(X_train_proc, y_train)
        print(f"🔧 [{method}] XGBoost trained.")
    else:
        raise NotImplementedError(method)

    # -------------------------
    # Internal validation
    # -------------------------
    preds_val = model.predict(X_val_proc)
    preds_val_rounded = np.clip(np.round(preds_val), 0, 10).astype(int)

    df_internal = pd.DataFrame({
        label_col: y_val,
        "predicted_match_score": preds_val_rounded
    })

    acc1_int, acc2_int, acc3_int = compute_accuracy(df_internal, label_col, "predicted_match_score")
    # mse_int = mean_squared_error(y_val, preds_val)
    # r2_int = r2_score(y_val, preds_val)
    mae_int = np.abs(y_val - preds_val_rounded).mean()  # ✅ Thêm MAE sau rounding

    print(f"📊 [{method}] Internal → Acc@1={acc1_int:.4f} Acc@2={acc2_int:.4f} Acc@3={acc3_int:.4f} MAE={mae_int:.4f}")

    plot_score_distribution(df_internal, method, chart_folder, label_col, "predicted_match_score", suffix="internal")

    # -------------------------
    # External test
    # -------------------------
    preds_ext = model.predict(X_external_proc)
    preds_ext_rounded = np.clip(np.round(preds_ext), 0, 10).astype(int)

    df_external = pd.DataFrame({
        label_col: y_external,
        "predicted_match_score": preds_ext_rounded
    })

    acc1_ext, acc2_ext, acc3_ext = compute_accuracy(df_external, label_col, "predicted_match_score")
    mse_ext = mean_squared_error(y_external, preds_ext)
    r2_ext = r2_score(y_external, preds_ext)
    mae_ext = np.abs(y_external - preds_ext_rounded).mean()  # ✅ Thêm MAE

    print(f"📊 [{method}] External → Acc@1={acc1_ext:.4f} Acc@2={acc2_ext:.4f} Acc@3={acc3_ext:.4f}  MSE={mse_ext:.4f} R2={r2_ext:.4f} MAE={mae_ext:.4f}")

    plot_score_distribution(df_external, method, chart_folder, label_col, "predicted_match_score", suffix="external")

    return {
        "model": model,
        "scaler": scaler,
        "pca": pca,
        "df_internal_out": df_internal,
        "df_external_out": df_external,
        "metrics_internal": {"Acc@1": acc1_int, "Acc@2": acc2_int, "Acc@3": acc3_int, "MAE": mae_int},
        "metrics_external": {"Acc@1": acc1_ext, "Acc@2": acc2_ext, "Acc@3": acc3_ext, "MAE": mae_ext},
    }



# -------------------------
# Runner
# -------------------------
def run_pipeline_all(df_train_embed,
                     df_test_embed,
                     method_list=None,
                     emb_col="combined_embedding",
                     label_col="match_score",
                     output_csv="./res_output_merge/metrics_summary.csv",
                     chart_folder="./res_output_merge/charts",
                     test_size=0.2,
                     random_state=42,
                     n_components=200):
    if method_list is None:
        method_list = ["pca_ridge", "ridge", "pca_lasso", "pca_svr"]

    os.makedirs(os.path.dirname(output_csv), exist_ok=True)
    os.makedirs(chart_folder, exist_ok=True)

    records = []
    for method in method_list:
        print("\n" + "="*60)
        try:
            res = train_and_eval(df_train_embed, df_test_embed,
                                 method=method,
                                 emb_col=emb_col,
                                 label_col=label_col,
                                 test_size=test_size,
                                 random_state=random_state,
                                 n_components=n_components,
                                 chart_folder=chart_folder)
            m_int = res["metrics_internal"]
            m_ext = res["metrics_external"]
            records.append({
                "method": method,
                "Acc@1_internal": m_int["Acc@1"], "Acc@2_internal": m_int["Acc@2"], "Acc@3_internal": m_int["Acc@3"], "MAE_internal": m_int["MAE"],
                "Acc@1_external": m_ext["Acc@1"], "Acc@2_external": m_ext["Acc@2"], "Acc@3_external": m_ext["Acc@3"], "MAE_external": m_ext["MAE"],
            })

        except Exception as e:
            print(f"❌ Method {method} failed: {e}")
            records.append({"method": method, **{k: np.nan for k in [
                "Acc@1_internal", "Acc@2_internal", "Acc@3_internal",
                "Acc@1_external", "Acc@2_external", "Acc@3_external"]}})

    df_summary = pd.DataFrame(records)
    df_summary.to_csv(output_csv, index=False)
    print(f"\n✅ Saved metrics summary to {output_csv}")
    return df_summary


# -------------------------
# Example usage
# -------------------------

import ast
import numpy as np

def parse_embedding(x):
    # Nếu đã là numpy array → giữ nguyên
    if isinstance(x, np.ndarray):
        return x.astype(float)
    # Nếu là list → convert sang np.array
    if isinstance(x, list):
        return np.array(x, dtype=float)
    # Nếu là chuỗi → parse sang list rồi np.array
    if isinstance(x, str):
        try:
            arr = ast.literal_eval(x)
            return np.array(arr, dtype=float)
        except Exception as e:
            raise ValueError(f"Cannot parse embedding string: {x[:100]}... ({e})")
    raise ValueError(f"Unsupported type for embedding: {type(x)}")

# --- Áp dụng cho train & test ---
for name, df in [("train", df_train_embed), ("test", df_test_embed)]:
    df["combined_embedding"] = df["combined_embedding"].apply(parse_embedding)
    print(f"✅ Parsed embeddings for {name} — sample shape:", df["combined_embedding"].iloc[0].shape)


method_list = ["pca_ridge", "ridge", "pca_lasso", "pca_svr"]
df_metrics = run_pipeline_all(df_train_embed, df_test_embed, method_list=method_list)

✅ Parsed embeddings for train — sample shape: (384,)
✅ Parsed embeddings for test — sample shape: (384,)

🔷 [pca_ridge] PCA n_components=200 explained_var_sum=0.9706
🔧 [pca_ridge] Ridge trained. alpha=100.0
📊 [pca_ridge] Internal → Acc@1=0.5114 Acc@2=0.7943 Acc@3=0.9357 MAE=1.6150
✅ Saved chart to ./res_output_merge/charts/pca_ridge_internal.png
📊 [pca_ridge] External → Acc@1=0.5437 Acc@2=0.8040 Acc@3=0.9390  MSE=3.6383 R2=0.3212 MAE=1.5403
✅ Saved chart to ./res_output_merge/charts/pca_ridge_external.png

🔧 [ridge] Ridge trained. alpha=31.622776601683793
📊 [ridge] Internal → Acc@1=0.5771 Acc@2=0.8471 Acc@3=0.9571 MAE=1.4179
✅ Saved chart to ./res_output_merge/charts/ridge_internal.png
📊 [ridge] External → Acc@1=0.5757 Acc@2=0.8337 Acc@3=0.9543  MSE=3.1956 R2=0.4038 MAE=1.4203
✅ Saved chart to ./res_output_merge/charts/ridge_external.png

🔷 [pca_lasso] PCA n_components=200 explained_var_sum=0.9706
🔧 [pca_lasso] Lasso trained. alpha=0.0036371550588259836
📊 [pca_lasso] Internal → Acc@1=0

# 4.Evaluate on test set

In [ ]:
df_real_embed = df_real_embed.rename(columns={'Score': 'match_score'})

df_real_embed

,CV_Text,JD_Text,match_score,combined_embedding
0,TRẦN HOÀN ĐỨC DUY\n& 0888 193 215 :unselected:...,Scientists ZAP advise business stakeholders on...,6,"[-0.057775355875492096, -0.003755283309146762,..."
1,TRẦN HOÀN ĐỨC DUY\n& 0888 193 215 :unselected:...,AI Engineer (AI Solution Team)\nFPT Smart Clou...,6,"[-0.028071988373994827, -0.04387618228793144, ..."
2,TRẦN HOÀN ĐỨC DUY\n& 0888 193 215 :unselected:...,Position summary\nIndustry: IT & Internet\nJob...,3,"[-0.047819338738918304, -0.01788148283958435, ..."
3,TRẦN HOÀN ĐỨC DUY\n& 0888 193 215 :unselected:...,Kỹ sư Dữ liệu lớn (Big Data Engineer)\n1. MÔ T...,3,"[-0.11163052171468735, 0.04298354685306549, 0...."
4,TRẦN HOÀN ĐỨC DUY\n& 0888 193 215 :unselected:...,Job description\nWe are looking for a Data Eng...,3,"[-0.0251266248524189, 0.03949412703514099, -0...."
...,...,...,...,...
82,"NGUYEN UT THIEN BACKEND NODEJS\nMAY 22, 1998\n...",Job description\nAbout the job\n\nFast and inn...,10,"[-0.055324096232652664, -0.06262123584747314, ..."
83,"NGUYEN UT THIEN BACKEND NODEJS\nMAY 22, 1998\n...",Mô tả công việc\n- Chào mời & hướng dẫn chủ cử...,0,"[-0.08064166456460953, 0.011131036095321178, 0..."
84,LIÊN HỆ\n0767506071\ntophuocthanh2@gmail.com\n...,Mô tả công việc\n- Chào mời & hướng dẫn chủ cử...,10,"[-0.08064166456460953, 0.011131036095321178, 0..."
85,Ninh Hoang Khai FRESHER FRONTEND DEVELOPER\nnh...,Job description\nAbout the job\n\nFast and inn...,8,"[-0.055324096232652664, -0.06262123584747314, ..."


In [ ]:
for name, df in [("real", df_real_embed)]:
    df["combined_embedding"] = df["combined_embedding"].apply(parse_embedding)
    print(f"✅ Parsed embeddings for {name} — sample shape:", df["combined_embedding"].iloc[0].shape)

df_metrics = run_pipeline_all(df_train_embed, df_real_embed, method_list=method_list)

✅ Parsed embeddings for real — sample shape: (384,)

🔷 [pca_ridge] PCA n_components=200 explained_var_sum=0.9706
🔧 [pca_ridge] Ridge trained. alpha=100.0
📊 [pca_ridge] Internal → Acc@1=0.5114 Acc@2=0.7943 Acc@3=0.9357 MAE=1.6150
✅ Saved chart to ./res_output_merge/charts/pca_ridge_internal.png
📊 [pca_ridge] External → Acc@1=0.2759 Acc@2=0.4483 Acc@3=0.6092  MSE=23.5596 R2=-1.9840 MAE=3.4253
✅ Saved chart to ./res_output_merge/charts/pca_ridge_external.png

🔧 [ridge] Ridge trained. alpha=31.622776601683793
📊 [ridge] Internal → Acc@1=0.5771 Acc@2=0.8471 Acc@3=0.9571 MAE=1.4179
✅ Saved chart to ./res_output_merge/charts/ridge_internal.png
📊 [ridge] External → Acc@1=0.1724 Acc@2=0.4023 Acc@3=0.6092  MSE=82.6445 R2=-9.4675 MAE=3.7126
✅ Saved chart to ./res_output_merge/charts/ridge_external.png

🔷 [pca_lasso] PCA n_components=200 explained_var_sum=0.9706
🔧 [pca_lasso] Lasso trained. alpha=0.0036371550588259836
📊 [pca_lasso] Internal → Acc@1=0.5079 Acc@2=0.7971 Acc@3=0.9379 MAE=1.6157
✅ Save

In [ ]:
# paraphrase-multilingual-MiniLM-L12-v2
df_metrics = run_pipeline_all(df_train_embed, df_real_embed, method_list=method_list)


🔷 [pca_ridge] PCA n_components=200 explained_var_sum=0.9706
🔧 [pca_ridge] Ridge trained. alpha=100.0
📊 [pca_ridge] Internal → Acc@1=0.5114 Acc@2=0.7943 Acc@3=0.9357 MAE=1.6150
✅ Saved chart to ./res_output_merge/charts/pca_ridge_internal.png
📊 [pca_ridge] External → Acc@1=0.2759 Acc@2=0.4483 Acc@3=0.6092  MSE=23.5596 R2=-1.9840 MAE=3.4253
✅ Saved chart to ./res_output_merge/charts/pca_ridge_external.png

🔧 [ridge] Ridge trained. alpha=31.622776601683793
📊 [ridge] Internal → Acc@1=0.5771 Acc@2=0.8471 Acc@3=0.9571 MAE=1.4179
✅ Saved chart to ./res_output_merge/charts/ridge_internal.png
📊 [ridge] External → Acc@1=0.1724 Acc@2=0.4023 Acc@3=0.6092  MSE=82.6445 R2=-9.4675 MAE=3.7126
✅ Saved chart to ./res_output_merge/charts/ridge_external.png

🔷 [pca_lasso] PCA n_components=200 explained_var_sum=0.9706
🔧 [pca_lasso] Lasso trained. alpha=0.0036371550588259836
📊 [pca_lasso] Internal → Acc@1=0.5079 Acc@2=0.7971 Acc@3=0.9379 MAE=1.6157
✅ Saved chart to ./res_output_merge/charts/pca_lasso_inter

In [ ]:
df_metrics = run_pipeline_all(df_train_embed, df_real_embed, method_list=method_list)


🔷 [pca_ridge] PCA n_components=200 explained_var_sum=0.9706
🔧 [pca_ridge] Ridge trained. alpha=100.0
📊 [pca_ridge] Internal → Acc@1=0.5114 Acc@2=0.7943 Acc@3=0.9357 MAE=1.6150
✅ Saved chart to ./res_output_merge/charts/pca_ridge_internal.png
📊 [pca_ridge] External → Acc@1=0.2759 Acc@2=0.4483 Acc@3=0.6092  MSE=23.5596 R2=-1.9840 MAE=3.4253
✅ Saved chart to ./res_output_merge/charts/pca_ridge_external.png

🔧 [ridge] Ridge trained. alpha=31.622776601683793
📊 [ridge] Internal → Acc@1=0.5771 Acc@2=0.8471 Acc@3=0.9571 MAE=1.4179
✅ Saved chart to ./res_output_merge/charts/ridge_internal.png
📊 [ridge] External → Acc@1=0.1724 Acc@2=0.4023 Acc@3=0.6092  MSE=82.6445 R2=-9.4675 MAE=3.7126
✅ Saved chart to ./res_output_merge/charts/ridge_external.png

🔷 [pca_lasso] PCA n_components=200 explained_var_sum=0.9706
🔧 [pca_lasso] Lasso trained. alpha=0.0036371550588259836
📊 [pca_lasso] Internal → Acc@1=0.5079 Acc@2=0.7971 Acc@3=0.9379 MAE=1.6157
✅ Saved chart to ./res_output_merge/charts/pca_lasso_inter